# Module 07: Advanced Patterns & Optimization - Follow Along

**Key Topics:** Circuit breakers, Caching, Profiling, Cost optimization

Run advanced optimization examples.


In [ ]:
%pip install -q -U pybreaker

from pybreaker import CircuitBreaker
import time
import hashlib

print('✅ Optimization tools ready!')

## Example 1: Circuit Breaker Pattern


In [ ]:
# Prevent cascade failures
breaker = CircuitBreaker(fail_max=3, timeout_duration=60)

@breaker
def call_external_api():
    """External API protected by circuit breaker."""
    import random
    if random.random() < 0.7:  # 70% failure rate
        raise Exception('API Error')
    return {'status': 'success'}

# Test
print("Testing circuit breaker:")
for i in range(10):
    try:
        result = call_external_api()
        print(f"Request {i+1}: ✅ Success")
    except Exception as e:
        print(f"Request {i+1}: ❌ {e}")
    time.sleep(0.1)

print(f"\nCircuit state: {breaker.current_state}")
print('✅ Circuit opens after 3 failures!')

## Example 2: Response Caching


In [ ]:
# LLM response caching
cache = {}  # In production: use Redis

def cached_llm_call(prompt: str) -> str:
    # Cache key
    cache_key = f"llm:{hashlib.md5(prompt.encode()).hexdigest()}"
    
    # Check cache
    if cache_key in cache:
        print(f"✅ Cache HIT for: {prompt[:30]}...")
        return cache[cache_key]
    
    # Call LLM (simulated)
    print(f"💰 Cache MISS - calling LLM for: {prompt[:30]}...")
    time.sleep(0.5)  # Simulate latency
    result = f"Response to: {prompt}"
    
    # Cache result
    cache[cache_key] = result
    return result

# Test
prompts = [
    'What is LangGraph?',
    'Explain agents',
    'What is LangGraph?',  # Hit
    'Explain agents'       # Hit
]

for prompt in prompts:
    result = cached_llm_call(prompt)

print(f'\n📊 Cache: {len(cache)} entries')
print('✅ 50% cache hit rate - significant cost savings!')

## Example 3: Performance Profiling


In [ ]:
import cProfile
import pstats
from io import StringIO

def slow_function():
    total = 0
    for i in range(1000000):
        total += i
    return total

def fast_function():
    return sum(range(1000000))

# Profile
profiler = cProfile.Profile()
profiler.enable()

slow_result = slow_function()
fast_result = fast_function()

profiler.disable()

# Print stats
s = StringIO()
ps = pstats.Stats(profiler, stream=s).sort_stats('cumulative')
ps.print_stats(5)

print("Top 5 time-consuming functions:")
print(s.getvalue()[:500])  # First 500 chars
print('\n✅ Identify bottlenecks with profiling!')

## Example 4: Cost Tracking


In [ ]:
class CostTracker:
    def __init__(self):
        self.total_tokens = 0
        self.total_cost = 0
        self.requests = 0
        # GPT-4 pricing
        self.cost_per_1k_tokens = 0.03
    
    def track(self, prompt_tokens: int, completion_tokens: int):
        tokens = prompt_tokens + completion_tokens
        cost = (tokens / 1000) * self.cost_per_1k_tokens
        
        self.total_tokens += tokens
        self.total_cost += cost
        self.requests += 1
        return cost
    
    def report(self):
        print(f"📊 Cost Report:")
        print(f"  Requests: {self.requests}")
        print(f"  Total tokens: {self.total_tokens:,}")
        print(f"  Total cost: ${self.total_cost:.4f}")
        print(f"  Avg cost/request: ${self.total_cost/max(1, self.requests):.4f}")

tracker = CostTracker()

# Simulate requests
requests = [
    {'prompt': 500, 'completion': 200},
    {'prompt': 300, 'completion': 150},
    {'prompt': 1000, 'completion': 500}
]

for req in requests:
    cost = tracker.track(req['prompt'], req['completion'])
    print(f"Request cost: ${cost:.4f}")

print()
tracker.report()
print('\n✅ Track costs to optimize spending!')

## Example 5: Exponential Backoff


In [ ]:
import random

def exponential_backoff_retry(func, max_retries=3):
    """Retry with exponential backoff."""
    for attempt in range(max_retries):
        try:
            return func()
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            
            wait_time = (2 ** attempt) + random.uniform(0, 1)
            print(f"Attempt {attempt+1} failed: {e}")
            print(f"Retrying in {wait_time:.2f}s...")
            time.sleep(wait_time)

def unreliable_function():
    if random.random() < 0.7:
        raise Exception('Temporary error')
    return 'Success!'

# Test
try:
    result = exponential_backoff_retry(unreliable_function)
    print(f"✅ {result}")
except Exception as e:
    print(f"❌ Final failure: {e}")

print('✅ Graceful error handling with backoff!')

## Summary

You've run examples for:
- ✅ Circuit breakers for resilience
- ✅ Response caching for cost savings
- ✅ Performance profiling
- ✅ Cost tracking and reporting
- ✅ Exponential backoff retry

**Next:** `module-07-practice.ipynb` for optimization exercises!
